# 🌧️ Sri Lanka Rainfall & Atmospheric Data (CHIRPS + ERA5 via Earth Engine)

In [4]:
# Install Earth Engine and Geemap
!pip install earthengine-api geemap --quiet


In [14]:
import ee
import geemap
import numpy as np


# Authenticate and initialize Earth Engine with project ID
ee.Authenticate()
ee.Initialize(project='lkenv2')

In [15]:
# Config
start_date = '2022-05-01'
end_date = '2022-05-03'
region = ee.Geometry.Rectangle([79.5, 5.5, 82.0, 10.0])
output_csv = f"/content/{start_date}_{end_date}_weather.csv"

# Dates list
dates = pd.date_range(start_date, end_date).strftime("%Y-%m-%d").tolist()

# Result container
records = []

# Loop over each date
for date in dates:
    try:
        # GFS: Use only first forecast of that day (e.g., 00 UTC F000–F003)
        gfs_img = ee.ImageCollection("NOAA/GFS0P25") \
            .filterDate(date, pd.to_datetime(date) + pd.Timedelta(days=1)) \
            .filter(ee.Filter.stringContains('system:index', 'F003')) \
            .first().clip(region)

        # CHIRPS rainfall
        chirps_img = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY") \
            .filterDate(date) \
            .first().clip(region)

        # Select only the valid bands
        gfs_img = gfs_img.select([
            'temperature_2m_above_ground',
            'relative_humidity_2m_above_ground',
            'u_component_of_wind_10m_above_ground',
            'v_component_of_wind_10m_above_ground'
        ])
        rainfall_img = chirps_img.rename('rainfall_mm')

        # Combine
        image = gfs_img.addBands(rainfall_img)

        # Reduce to regional mean
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=region,
            scale=25000,
            maxPixels=1e8
        ).getInfo()

        temp_k = stats['temperature_2m_above_ground']
        rh = stats['relative_humidity_2m_above_ground']
        temp_c = temp_k - 273.15

        # Compute dewpoint
        gamma = np.log(rh / 100.0) + (17.625 * temp_c) / (243.04 + temp_c)
        dewpoint_c = (243.04 * gamma) / (17.625 - gamma)
        dewpoint_k = dewpoint_c + 273.15

        records.append({
            "date": date,
            "city": "Sri Lanka (mean)",
            "latitude": 7.87,
            "longitude": 80.7,
            "dewpoint_temperature_2m": dewpoint_k,
            "rainfall_mm": stats.get('rainfall_mm'),
            "surface_pressure": "",  # Not in GFS directly
            "temperature_2m": temp_k,
            "u_component_of_wind_10m": stats.get('u_component_of_wind_10m_above_ground'),
            "v_component_of_wind_10m": stats.get('v_component_of_wind_10m_above_ground'),
            "temperature_2m_c": temp_c,
            "dewpoint_temperature_2m_c": dewpoint_c,
            "humidity": rh
        })

    except Exception as e:
        print(f"❌ Skipped {date}: {str(e)}")

# Save CSV
df = pd.DataFrame(records)
df.to_csv(output_csv, index=False)
print(f"\n✅ Weather data saved: {output_csv}")
df.head()



✅ Weather data saved: /content/2022-05-01_2022-05-03_weather.csv


,date,city,latitude,longitude,dewpoint_temperature_2m,rainfall_mm,surface_pressure,temperature_2m,u_component_of_wind_10m,v_component_of_wind_10m,temperature_2m_c,dewpoint_temperature_2m_c,humidity
0,2022-05-01,Sri Lanka (mean),7.87,80.7,28.928011,5.589264,,28.928100,4.072394,1.947622,-244.221900,-244.221989,76.315357
1,2022-05-02,Sri Lanka (mean),7.87,80.7,29.036076,3.179267,,29.036154,3.802904,1.467789,-244.113846,-244.113924,74.959873
2,2022-05-03,Sri Lanka (mean),7.87,80.7,28.906113,3.633942,,28.906199,2.430756,1.274213,-244.243801,-244.243887,77.377642


In [ ]:
# ReliefWeb Text Data for Sri Lanka Disasters

!pip install requests --quiet

import requests
import pandas as pd

# 1) Fetch disaster events
url = "https://api.reliefweb.int/v1/disasters"
params = {
    'appname': 'my-ai-project',
    'filter[field]': 'country.primary',
    'filter[value]': 'Sri Lanka',
    'profile': 'full',
    'limit': 50
}
dis = requests.get(url, params=params).json()

events = []
for d in dis['data']:
    fid = d['id']
    name = d['fields']['name']
    date = d['fields']['date']['created'][0:10]
    events.append({'disaster_id': fid, 'name': name, 'date': date})
df_dis = pd.DataFrame(events)

# 2) Fetch reports for a selected disaster
if not df_dis.empty:
    did = df_dis.iloc[0]['disaster_id']
    r_url = "https://api.reliefweb.int/v1/reports"
    r_params = {
        'appname': 'my-ai-project',
        'filter[field]': 'disaster.id',
        'filter[value]': did,
        'profile': 'full',
        'limit': 20
    }
    rep = requests.get(r_url, params=r_params).json()
    reports = []
    for repi in rep['data']:
        f = repi['fields']
        reports.append({
            'id': repi['id'],
            'date': f['date']['created'][0:10],
            'source': f['source']['name'],
            'title': f['title'],
            'body': f.get('body', '')
        })
    df_reports = pd.DataFrame(reports)
    display(df_reports[['date','source','title']])
else:
    print("No disaster events found.")


KeyError: 'data'